In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from numba import jit
from scipy.interpolate import interp1d
import ipywidgets as widgets
from ipywidgets import interactive, VBox
from IPython.display import display

# ---- BFS for 2D ----
@jit(nopython=True)
def percolation_bfs_2D(grid):
    grid_size_row, grid_size_col = grid.shape
    visited = np.zeros_like(grid, dtype=np.bool_)
    queue = np.zeros((grid_size_row * grid_size_col, 2), dtype=np.int32)

    front, rear = -1, -1
    for i in range(grid_size_col):
        if not grid[0, i]:
            rear += 1
            queue[rear] = (0, i)
            visited[0, i] = True

    while front != rear:
        front += 1
        row, col = queue[front]

        for dr, dc in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
            nr, nc = row + dr, col + dc
            if 0 <= nr < grid_size_row and 0 <= nc < grid_size_col and not grid[nr, nc] and not visited[nr, nc]:
                rear += 1
                queue[rear] = (nr, nc)
                visited[nr, nc] = True

    return int(np.any(visited[-1]))

# ---- BFS for 3D ----
@jit(nopython=True)
def percolation_bfs_3D(grid):
    grid_size_row, grid_size_col, grid_size_depth = grid.shape
    visited = np.zeros_like(grid, dtype=np.bool_)
    queue = np.zeros((grid_size_row * grid_size_col * grid_size_depth, 3), dtype=np.int32)

    front, rear = -1, -1
    for i in range(grid_size_col):
        for j in range(grid_size_depth):
            if not grid[0, i, j]:
                rear += 1
                queue[rear] = (0, i, j)
                visited[0, i, j] = True

    while front != rear:
        front += 1
        row, col, depth = queue[front]

        for dr, dc, dd in [(1, 0, 0), (0, 1, 0), (-1, 0, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]:
            nr, nc, nd = row + dr, col + dc, depth + dd
            if 0 <= nr < grid_size_row and 0 <= nc < grid_size_col and 0 <= nd < grid_size_depth:
                if not grid[nr, nc, nd] and not visited[nr, nc, nd]:
                    rear += 1
                    queue[rear] = (nr, nc, nd)
                    visited[nr, nc, nd] = True

    return int(np.any(visited[-1]))

# ---- Percolation Simulation ----
def estimate_percolation_threshold(grid_size_row, grid_size_col, grid_size_depth, trials=100, p_resolution=1001, is_3D=False, model_type="2D", z=4):
    p_values = np.linspace(0, 1, p_resolution)
    percolation_probabilities = np.zeros_like(p_values)

    start_time = time.time()

    for i, p in enumerate(p_values):
        success_count = sum(
            percolation_bfs_3D(np.random.rand(grid_size_row, grid_size_col, grid_size_depth) > p) if is_3D
            else percolation_bfs_2D(np.random.rand(grid_size_row, grid_size_col) > p)
            for _ in range(trials)
        )
        percolation_probabilities[i] = success_count / trials

    elapsed_time = time.time() - start_time
    print(f'Time taken: {elapsed_time:.2f} seconds')

    try:
        interp_func = interp1d(percolation_probabilities, p_values, kind='linear', fill_value='extrapolate')
        percolation_threshold = interp_func(0.5)
    except ValueError:
        percolation_threshold = np.nan
        print("Interpolation error: Not enough variation in percolation probabilities. Try increasing trials or adjusting grid size.")

    title_text = f'Model: {model_type}, Coordination Number (z): {z}'

    plt.figure(figsize=(6, 4))
    plt.plot(p_values, percolation_probabilities, label="Percolation Probability")
    plt.xlabel('Conducting site fraction')
    plt.ylabel('Percolation probability')
    plt.title(title_text)
    plt.grid(True)

    if not np.isnan(percolation_threshold):
        plt.axvline(x=percolation_threshold, color='r', linestyle='--', label=f'Percolation Threshold: {percolation_threshold:.3f}')

    plt.legend()
    plt.show()

# ---- Interactive UI ----
def interactive_simulation(model_type, z, grid_size_row, grid_size_col, grid_size_depth, trials, p_resolution):
    is_3D = model_type == "3D"
    estimate_percolation_threshold(grid_size_row, grid_size_col, grid_size_depth, trials, p_resolution, is_3D, model_type, z)

def update_widgets(change):
    if model_type_widget.value == "2D":
        z_widget.options = [4, 8]
        grid_size_depth_widget.disabled = True
    else:
        z_widget.options = [6, 26]
        grid_size_depth_widget.disabled = False

# ---- Widgets ----
model_type_widget = widgets.Dropdown(options=["2D", "3D"], value="2D", description="Model Type:")
z_widget = widgets.Dropdown(options=[4, 8], value=4, description="z:")
model_type_widget.observe(update_widgets, names='value')

grid_size_row_widget = widgets.IntSlider(value=10, min=5, max=100, step=1, description="Lx:")
grid_size_col_widget = widgets.IntSlider(value=10, min=5, max=100, step=1, description="Ly:")
grid_size_depth_widget = widgets.IntSlider(value=10, min=5, max=100, step=1, description="Lz:", disabled=True)

trials_widget = widgets.IntSlider(value=100, min=10, max=500, step=10, description="Trials:")
p_resolution_widget = widgets.IntSlider(value=1001, min=100, max=5000, step=100, description="p resolution:")

interactive_plot = interactive(interactive_simulation,
                               model_type=model_type_widget,
                               z=z_widget,
                               grid_size_row=grid_size_row_widget,
                               grid_size_col=grid_size_col_widget,
                               grid_size_depth=grid_size_depth_widget,
                               trials=trials_widget,
                               p_resolution=p_resolution_widget)

# ---- Initialize widget state ----
update_widgets(None)

# ---- Display in notebook ----
if __name__ == "__main__":
    display(interactive_plot)


interactive(children=(Dropdown(description='Model Type:', options=('2D', '3D'), value='2D'), Dropdown(descript…

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from numba import jit, prange
import ipywidgets as widgets
from ipywidgets import interactive
from scipy.interpolate import interp1d
from IPython.display import display

# ---- Node Generator ----
def generate_fixed_structure(grid_size_row, grid_size_col, grid_size_depth):
    num_nodes = grid_size_row * grid_size_col * grid_size_depth
    all_nodes = np.random.rand(num_nodes, 3) * np.array([grid_size_row, grid_size_col, grid_size_depth])
    return all_nodes

# ---- Connectivity Matrix ----
@jit(nopython=True, parallel=True)
def compute_connectivity_matrix(all_nodes, threshold_distance):
    num_nodes = all_nodes.shape[0]
    connectivity_matrix = np.zeros((num_nodes, num_nodes), dtype=np.bool_)
    coordination_numbers = np.zeros(num_nodes, dtype=np.int32)

    for i in prange(num_nodes):
        for j in range(i + 1, num_nodes):
            dist = np.linalg.norm(all_nodes[i] - all_nodes[j])
            if dist <= threshold_distance:
                connectivity_matrix[i, j] = True
                connectivity_matrix[j, i] = True
                coordination_numbers[i] += 1
                coordination_numbers[j] += 1

    avg_coordination_number = np.mean(coordination_numbers)
    return connectivity_matrix, avg_coordination_number

# ---- BFS Algorithm ----
@jit(nopython=True)
def percolation_bfs(connectivity_matrix, num_conductive_nodes, disabled_nodes):
    num_nodes = connectivity_matrix.shape[0]
    active_nodes = np.ones(num_nodes, dtype=np.bool_)
    active_nodes[disabled_nodes] = False

    visited = np.zeros(num_nodes, dtype=np.bool_)
    queue = np.full(num_nodes, -1, dtype=np.int32)
    front, rear = -1, -1

    sorted_indices = np.argsort(np.sum(connectivity_matrix, axis=1))
    start_nodes = sorted_indices[:max(1, int(np.ceil(num_conductive_nodes * 0.1)))]
    end_nodes = sorted_indices[-max(1, int(np.ceil(num_conductive_nodes * 0.1))):]

    for start in start_nodes:
        if active_nodes[start]:
            rear += 1
            queue[rear] = start
            visited[start] = True

    while front != rear:
        front += 1
        current_node = queue[front]

        if current_node in end_nodes:
            return 1

        for neighbor in range(num_nodes):
            if connectivity_matrix[current_node, neighbor] and not visited[neighbor] and active_nodes[neighbor]:
                rear += 1
                queue[rear] = neighbor
                visited[neighbor] = True

    return 0

# ---- Percolation Simulation ----
def estimate_percolation_threshold(grid_size_row, grid_size_col, grid_size_depth,
                                    threshold_distance, trials=100, p_resolution=101):
    print("Running Random Lattice Percolation Simulation...")

    all_nodes = generate_fixed_structure(grid_size_row, grid_size_col, grid_size_depth)
    connectivity_matrix, avg_coordination_number = compute_connectivity_matrix(all_nodes, threshold_distance)

    p_values = np.linspace(0, 1, p_resolution)
    percolation_probabilities = np.zeros_like(p_values)
    num_nodes = connectivity_matrix.shape[0]

    start_time = time.time()

    for i, p in enumerate(p_values):
        num_conductive_nodes = int(np.floor(num_nodes * p))
        if num_conductive_nodes < 1:
            continue

        success_count = 0
        for _ in range(trials):
            disabled_nodes = np.random.choice(num_nodes, num_nodes - num_conductive_nodes, replace=False)
            success_count += percolation_bfs(connectivity_matrix, num_conductive_nodes, disabled_nodes)

        percolation_probabilities[i] = success_count / trials

    elapsed_time = time.time() - start_time
    print(f'Time taken: {elapsed_time:.2f} seconds')

    try:
        interp_func = interp1d(percolation_probabilities, p_values, kind='linear', fill_value='extrapolate')
        percolation_threshold = interp_func(0.5)
    except ValueError:
        percolation_threshold = np.nan
        print("Interpolation error: Not enough variation. Try more trials or better parameters.")

    plt.figure(figsize=(6, 4))
    plt.plot(p_values, percolation_probabilities, label="Percolation Probability")
    plt.xlabel('Conducting site fraction')
    plt.ylabel('Percolation probability')
    plt.title(f'Model: Random, Average Coordination Number (z): {avg_coordination_number:.2f}')
    plt.grid(True)

    if not np.isnan(percolation_threshold):
        plt.axvline(x=percolation_threshold, color='r', linestyle='--',
                    label=f'Percolation Threshold: {percolation_threshold:.3f}')
    else:
        plt.axvline(x=0, color='r', linestyle='--', label='Percolation Threshold: N/A')

    plt.legend()
    plt.show()

# ---- Widgets ----
grid_size_row_widget = widgets.IntSlider(value=10, min=5, max=30, step=1, description="Lx:")
grid_size_col_widget = widgets.IntSlider(value=10, min=5, max=30, step=1, description="Ly:")
grid_size_depth_widget = widgets.IntSlider(value=10, min=5, max=30, step=1, description="Lz:")
threshold_distance_widget = widgets.FloatSlider(value=1.0, min=0.9, max=2.0, step=0.05, description="Dth:")
trials_widget = widgets.IntSlider(value=100, min=10, max=500, step=10, description="Trials:")
p_resolution_widget = widgets.IntSlider(value=101, min=100, max=5000, step=100, description="p resolution:")

interactive_plot = interactive(
    estimate_percolation_threshold,
    grid_size_row=grid_size_row_widget,
    grid_size_col=grid_size_col_widget,
    grid_size_depth=grid_size_depth_widget,
    threshold_distance=threshold_distance_widget,
    trials=trials_widget,
    p_resolution=p_resolution_widget
)

# ---- Display ----
if __name__ == "__main__":
    display(interactive_plot)


interactive(children=(IntSlider(value=10, description='Lx:', max=30, min=5), IntSlider(value=10, description='…